# Beta Decay Regression with PyTorch

This notebook uses PyTorch to do ML regression of the solution to the ODE system describing beta decay as a simple proxy for more complicated reaction networks.

The two-component system is:

$$\dot{X}_0 (t) = - A \cdot X_0 (t)$$
$$\dot{X}_1 (t) = + A \cdot X_0 (t)$$

This system has an analytic solution we can compare with the model predictions:

$$X_0 (t) = X_0 (0) \cdot e^{-A \cdot t}$$
$$X_1 (t) = X_1 (0) + X_0 (0) \cdot \left(1 - e^{-A \cdot t}\right)$$

For this example, the exponential decay constant $A$ and the initial conditions are set to:

$$A = 1$$
$$X_0 (0) = 1$$
$$X_1 (0) = 0$$

The model input is the time $t$ and the outputs are $X_0 (t)$ and $X_1 (t)$.

(Although the below is written as $y(x)$ instead of $x(t)$ unfortunately, maybe I'll eventually change it...)

## Module Imports

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import imageio

%matplotlib agg

In [6]:
import numpy as np

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

## Runtime Options

`use_cuda = [True] or [False]`

This notebook is set up to allow training on a GPU or CPU.

Because we are generating plot data within the training loop, GPU training isn't particularly efficient. But it is a reference to the PyTorch API.

`use_exact_model = [True] or [False]`

There are two models for training here:

1. A feed-forward multi-layer neural network

Number of independent variables (inputs), dependent variables (outputs), number of hidden layers, number of nodes per hidden layer, and activation function are customizable.

2. The analytic solution to the ODE system

The analytic solution to the ODE system is parameterized by the initial conditions and the exponential decay constant. Setting `use_exact_model=True` demonstrates that the backpropagation training scheme can use the parameterized ODE solution to fit the data.

In [8]:
# Set use_cuda=True to use an available GPU (for reference)
# This doesn't really help for this example though since making plots
# in each training iteration requires copying tensors to the CPU.
## NOTE: I've commented this out because a global variable in the notebook like this makes it so PyTorch cannot compile the model to TorchScript.
##       There are workarounds, like making the model construction & training a function, which we would use for production code.
use_cuda=False

In [9]:
# Set use_exact_model=True to use a model parameterized
# exactly like the analytic solution
use_exact_model=False

## Generate Training Data

### Number of Samples & timescale for input

In [10]:
# generate NumSamples for training
NumSamples = 128

# let's look at a range in x from [0, 10]
xmin = 0
xmax = 10.0

# the range of the solution values is [0,1]
ymin = 0
ymax = 1

x = torch.unsqueeze(torch.linspace(xmin, xmax, NumSamples, requires_grad=True), dim=1)
x_test = torch.unsqueeze(torch.rand(NumSamples, requires_grad=True), dim=1) * (xmax-xmin) + xmin

### Analytic Solution & RHS Function

In [11]:
# initial conditions at "time" 0
y0_t0 = 1
y1_t0 = 0

# exponential decay constant
exp_A = 1

# ya & yb are tensors that let us express the analytic
# solution of the system in matrix form
ya = torch.ones(1,2)
ya[0][0] =  y0_t0
ya[0][1] = -y0_t0

yb = torch.ones(1,2)
yb[0][0] = 0
yb[0][1] = y0_t0 + y1_t0

# this is the analytic solution of the system as a function of x
def sol(_x):
    return torch.exp(-exp_A * _x) * ya + yb

# this is the analytic derivative of the system w.r.t. x
def rhs(_y):
    return -exp_A * (_y - yb)

# if we're using cuda, then put the tensors
# in our workspace on the GPU.
#if use_cuda:
#    x = x.cuda()
#    x_test = x_test.cuda()
#    yb = yb.cuda()
#    ya = ya.cuda()

In [12]:
# get the analytic solution as a function of x
y = sol(x)

# get the analytic right-hand-side as a function of y(x)
# f(x) = dy(x)/dx
dydx = rhs(y)

# get the analytic solution at the test points x_test
y_test = sol(x_test)
    
# we will want to propagate gradients through y, dydx, and x
# so make them PyTorch Variables
x = Variable(x, requires_grad=True)
y = Variable(y, requires_grad=True)
dydx = Variable(dydx, requires_grad=True)

# we will need to evaluate gradients w.r.t. x multiple
# times so tell PyTorch to save the gradient variable in x.
x.retain_grad()

In [13]:
# get numpy versions of x,y,f on the cpu for plotting
xnp = x.cpu().data.numpy()
ynp = y.cpu().data.numpy()
fnp = dydx.cpu().data.numpy()

## Setting up the Models, Optimizers & Loss Function ...

In [14]:
class HiddenNet(nn.Module):
    def __init__(self, n_independent, n_dependent,
                 n_hidden, hidden_depth, activation):
        super(HiddenNet, self).__init__()
        
        self.activation = activation
        self.input_layer = nn.Linear(n_independent, n_hidden)
        self.hidden_layers = nn.ModuleList()
        for i in range(hidden_depth):
            self.hidden_layers.append(nn.Linear(n_hidden, n_hidden))
        self.output_layer = nn.Linear(n_hidden, n_dependent)
        
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        for h in self.hidden_layers:
            x = self.activation(h(x))
        x = self.output_layer(x)
        return x

In [15]:
class ExactModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.y0_0 = torch.nn.Parameter(torch.tensor(1.0))
        self.y1_0 = torch.nn.Parameter(torch.tensor(1.0))
        self.a = torch.nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        # ya & yb are tensors that let us express the analytic
        # solution of the system in matrix form
        ya = torch.ones(1,2)
        ya[0][0] =  self.y0_0
        ya[0][1] = -self.y0_0

        yb = torch.ones(1,2)
        yb[0][0] = 0
        yb[0][1] = self.y0_0 + self.y1_0
        
#        if use_cuda:
#            ya = ya.cuda()
#            yb = yb.cuda()

        return torch.exp(-self.a * x) * ya + yb

In [16]:
n_independent_inputs = 1
n_dependent_outputs = 2
n_hidden_layers = 2
hidden_layer_depth = 0
activation_function = F.celu

if use_exact_model:
    net = ExactModel()
else:
    net = HiddenNet(n_independent=n_independent_inputs, n_dependent=n_dependent_outputs,
                    n_hidden=n_hidden_layers, hidden_depth=hidden_layer_depth, activation=activation_function)

In [17]:
#if use_cuda:
#    net.cuda()

In [18]:
print(net)

HiddenNet(
  (input_layer): Linear(in_features=1, out_features=2, bias=True)
  (hidden_layers): ModuleList()
  (output_layer): Linear(in_features=2, out_features=2, bias=True)
)


In [19]:
optimizer_sgd = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer_adam = torch.optim.Adam(net.parameters(), lr=0.01)

In [20]:
loss_func = torch.nn.MSELoss()

## Training Loop and Plotting

In [21]:
training_images = []
# get a multipanel figure showing the prediction (p) and error (e)
fig, (axis_p, axis_f, axis_e) = plt.subplots(nrows=3, ncols=1, figsize=(8,8), dpi=150)
axis_e1 = axis_e.twinx()

In [22]:
# arrays for accumulating the epoch index and losses for plotting
epochs = []
losses = []
losses0 = []
losses1 = []
tlosses = []

In [23]:
# this function is the training loop over epochs
# where 1 epoch trains over the whole training dataset
def train_error(NumEpochs):
    for t in range(NumEpochs):
        # calculate prediction given the current net state
        prediction = net(x)
        
        # get error with testing samples
        prediction_test = net(x_test)
        test_loss = torch.sqrt(loss_func(prediction_test, y_test)).cpu().data.numpy()

        # calculate error between prediction and analytic truth y
        loss0 = torch.sqrt(loss_func(prediction, y))

        # calculate gradients d(prediction)/d(x) for each component

        # first, zero out the existing gradients to avoid
        # accumulating gradients on top of existing gradients
        net.zero_grad()

        if x.grad is not None:
            x.grad.data.zero_()

        # now get the gradients dp0/dx
        prediction[:,0].backward(torch.ones_like(prediction[:,0]), retain_graph=True)
        # clone the x gradient to save a copy of it as dp0/dx
        dp0dx = x.grad.clone()
        # clear the x gradient for the loss gradient below
        x.grad.data.zero_()
        
        # get gradient dp1/dx
        prediction[:,1].backward(torch.ones_like(prediction[:,1]), retain_graph=True)
        # clone the x gradient to save a copy of it as dp1/dx
        dp1dx = x.grad.clone()
        # clear the x gradient for the loss gradient below
        x.grad.data.zero_()
        
        dpdx = torch.ones_like(prediction)
        dpdx[:,0] = torch.flatten(dp0dx)
        dpdx[:,1] = torch.flatten(dp1dx)
        
        # evaluate the analytic right-hand-side function at the prediction value
        prhs = rhs(prediction)

        # define the error of the prediction derivative using the analytic derivative
        loss1 = torch.sqrt(loss_func(dpdx, dydx))
        
        # the following doesn't work well :/
        #loss1 = torch.sqrt(loss_func(dpdx, rhs(prediction)))

        # total error combines the error of the prediction (loss0) with 
        # the error of the prediction derivative (loss1)
        loss = loss0 + loss1

        # use the Adam optimizer
        optimizer = optimizer_adam

        # clear gradients for the next training iteration
        optimizer.zero_grad()

        # compute backpropagation gradients
        loss.backward()

        # apply gradients to update the weights
        optimizer.step()

        # generate plots
        if t % 10 == 0:
            # Prediction plot to show learning progress
            axis_p.clear()

            axis_p.set_ylabel('Solution', fontsize=22)

            pnp = prediction.cpu().data.numpy()
            
            axis_p.plot(xnp, pnp[:,0],
                        'orange', lw=3, alpha=0.5,
                        label='p(t)')
            
            axis_p.plot(xnp, pnp[:,1],
                        'red', lw=3, alpha=0.5)
            
            axis_p.scatter(xnp, ynp[:,0],
                           color='blue', alpha=0.5, s=20,
                           label='x(t)')

            axis_p.scatter(xnp, ynp[:,1],
                           color='green', alpha=0.5, s=20)
            
            axis_p.legend(loc='upper right', borderpad=1, framealpha=0.5)
            
            # Plot analytic rhs vs prediction rhs
            pfnp = prhs.cpu().data.numpy()
            dpdxnp = dpdx.cpu().data.numpy()
            axis_f.clear()

            axis_f.set_ylabel('Gradient', fontsize=22)
            
            axis_f.plot(xnp, pfnp[:,0],
                        'orange', lw=3, alpha=0.5,
                        label='f(p(t))')
            
            axis_f.plot(xnp, pfnp[:,1],
                        'red', lw=3, alpha=0.5)
            
            axis_f.plot(xnp, dpdxnp[:,0],
                        'magenta', lw=3, ls=':', alpha=0.5,
                        label='dp(t)/dt')
            
            axis_f.plot(xnp, dpdxnp[:,1],
                        'black', lw=3, ls=':', alpha=0.5)

            axis_f.scatter(xnp, fnp[:,0],
                           color='blue', alpha=0.5, s=20,
                           label='f(x(t))')

            axis_f.scatter(xnp, fnp[:,1],
                           color='green', alpha=0.5, s=20)
            
            axis_f.tick_params(axis='both', which='major', labelsize=16)
            
            axis_f.legend(loc='upper right', borderpad=1, framealpha=0.5)

            # get min/max in x/y to set label positions relative to the axes
            height = np.abs(ymax - ymin)
            width = np.abs(xmax - xmin)

            axis_p.set_xlim(xmin, xmax)
            axis_p.set_ylim(ymin, ymax)

            axis_p.text(xmin, ymax + height*0.3,
                      'Step = %d' % t, fontdict={'size': 24, 'color': 'blue'})
            axis_p.text(xmin + width*0.5, ymax + height*0.3,
                      'Train Loss = %.2e' % loss.cpu().data.numpy(),
                      fontdict={'size': 24, 'color': 'blue'})
            axis_p.text(xmin + width*0.5, ymax + height*0.1,
                      'Test Loss = %.2e' % test_loss,
                      fontdict={'size': 24, 'color': 'orange'})

            axis_p.tick_params(axis='both', which='major', labelsize=16)

            # plot errors evolving with the number of epochs trained
            epochs.append(t)
            losses.append(loss.cpu().data.numpy())
            losses0.append(loss0.cpu().data.numpy())
            losses1.append(loss1.cpu().data.numpy())
            tlosses.append(test_loss)

            axis_e.clear()
            axis_e1.clear()

            axis_e.set_xlabel('Epoch', fontsize=22)
            axis_e.set_ylabel('E(p,x)', fontsize=22)

            axis_e.scatter([epochs[-1]], [losses0[-1]],
                           color="red", alpha=0.5)
            axis_e.plot(epochs, losses0,
                        'b-', lw=3, alpha=0.5,
                        label='E(p,x) [train]')

            axis_e.scatter([epochs[-1]], [test_loss],
                           color="red", alpha=0.5)
            axis_e.plot(epochs, tlosses,
                        'orange', lw=3, ls="--", alpha=0.5,
                        label='E(p,x) [test]')

            axis_e1.set_ylabel('E(dp/dt, f(x))', fontsize=22)

            axis_e1.scatter([epochs[-1]], [losses1[-1]],
                           color="red", alpha=0.5)
            axis_e1.plot(epochs, losses1,
                         'g-', lw=3, alpha=0.5,
                         label='E(dp/dt, f(x)) [train]')
            
            axis_e.get_yaxis().set_major_formatter(
                matplotlib.ticker.FuncFormatter(lambda x, p: "{:0.1f}".format(x)))
            
            axis_e1.get_yaxis().set_major_formatter(
                matplotlib.ticker.FuncFormatter(lambda x, p: "{:0.1f}".format(x)))

            axis_e.tick_params(axis='both', which='major', labelsize=16)
            axis_e1.tick_params(axis='both', which='major', labelsize=16)
            
            axis_e.legend(loc='upper right', borderpad=1, framealpha=0.5)
            axis_e1.legend(loc='upper center', borderpad=1, framealpha=0.5)

            # Draw on canvas and save image in sequence
            fig.canvas.draw()
            plt.tight_layout()
            image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

            training_images.append(image)

            # Print epoch/error notifications
            if t % 100 == 0:
                print('epoch ', t, ' with error: ', losses[-1])
           
        # Stop early if our errors are plateauing
        if t > 1000:
            # do a quadratic polynomial fit and see if we will
            # need more than NumEpochs for the error e to vanish:
            # e / (d(e)/d(epoch)) > NumEpochs ?
            # if so, then break out of the training loop ...
            xfit = epochs[-4:]
            efit = losses[-4:]
            coef = np.polyfit(xfit, efit, 2)
            
            if coef[2]/coef[1] > NumEpochs:
                break

In [24]:
train_error(1000)

epoch  0  with error:  2.237783
epoch  100  with error:  0.354963
epoch  200  with error:  0.31930387
epoch  300  with error:  0.28354263
epoch  400  with error:  0.24477826
epoch  500  with error:  0.1411982
epoch  600  with error:  0.044161655
epoch  700  with error:  0.03513615
epoch  800  with error:  0.028354764
epoch  900  with error:  0.024161939


In [26]:
imageio.mimsave('./beta_decay.gif', training_images, duration=100)

In [ ]:
print("final test sample error: ", tlosses[-1])

## Save the Model File We Trained

This uses the just-in-time compiler to compile the network into TorchScript (which is portable across python/C++) and then saves that script.

In [ ]:
model_scripted = torch.jit.script(net)

model_scripted.save('beta_decay_model.pt')

## Load the Model We Trained

Load our just-in-time compiled TorchScript model.

In [ ]:
saved_model = torch.jit.load('beta_decay_model.pt')

Evaluate our loaded model and compare it to the model we originally saved

In [ ]:
eval_saved_model = saved_model(x)

In [ ]:
original_model = net(x)

In [ ]:
diff = eval_saved_model - original_model

And if all went well, the difference between the saved model and the model we finished training should be 0s everywhere when we evaluate it on the training data ...

In [ ]:
diff.data.numpy()